In [1]:
import torch
from torch_geometric.data import InMemoryDataset, download_url, Data, Batch
from torch import nn
from torch.nn import functional as F
import os
import pandas as pd
import numpy as np
import pickle
import itertools
import jax
from jax import numpy as jnp
import networkx as nx
from scipy.spatial.distance import pdist, squareform
from sklearn.preprocessing import MinMaxScaler
import mendeleev
import ray
from ray import tune
from ray.tune.schedulers import ASHAScheduler, HyperBandForBOHB
from ray.tune.suggest.bohb import TuneBOHB
import os

In [2]:
import os
from torch.utils.data import TensorDataset, DataLoader

In [3]:
files_train = os.listdir("/mnt/10edb508-27ad-4f92-9467-37a536784b53/temp/train/")

In [4]:
data = []
labels = []
directory = "/mnt/10edb508-27ad-4f92-9467-37a536784b53/temp/train/"
files_train = os.listdir(directory)
for file in np.sort(files_train):
    path = os.path.join(directory, file)
    if file[0] == "d":
        data.append(pd.read_hdf(path))
    else:
        labels.append(pd.read_hdf(path))

In [5]:
y_train = pd.concat(labels)
X_train = pd.concat(data)

In [6]:
data = []
labels = []
directory = "/mnt/10edb508-27ad-4f92-9467-37a536784b53/temp/test/"
files_test = os.listdir(directory)
for file in np.sort(files_test):
    path = os.path.join(directory, file)
    if file[0] == "d":
        data.append(pd.read_hdf(path))
    else:
        labels.append(pd.read_hdf(path))

In [7]:
X_temp_train = X_train[(y_train[0] == 3).to_numpy()]
y_temp_train = y_train[(y_train[0] == 3).to_numpy()]

In [8]:
y_test = pd.concat(labels)
X_test = pd.concat(data)

In [10]:
X_temp_test = X_test[(y_test[0] == 3).to_numpy()]
y_temp_test = y_test[(y_test[0] == 3).to_numpy()]

In [11]:
train_dataset = TensorDataset(torch.Tensor(X_temp_train.to_numpy()),torch.Tensor(y_temp_train.to_numpy())) # create your datset
train_dataloader = DataLoader(train_dataset, num_workers=12, batch_size=4096, shuffle=True, drop_last=True)
test_dataset = TensorDataset(torch.Tensor(X_temp_test.to_numpy()),torch.Tensor(y_temp_test.to_numpy())) # create your datset
test_dataloader = DataLoader(test_dataset, num_workers=12, batch_size=4096, shuffle=True)

In [12]:
del X_train, X_test, data, labels

In [16]:
def init_weights(m):
    if isinstance(m, nn.Linear):
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.01)

class ResNetGated(nn.Module):
    def __init__(self, init_dim, hidden_dim, layers, p_dropout):
        super().__init__()
        self.p_dropout = p_dropout
        assert layers > 0
        self.layers = nn.ModuleList([nn.Sequential(nn.Linear(init_dim, hidden_dim),
                             nn.ReLU(),
                             nn.BatchNorm1d(hidden_dim),
                             nn.Linear( hidden_dim, init_dim)) for i in range(layers)])
        self.gates = nn.Parameter(torch.Tensor(layers))
        self.layers.apply(init_weights)
    def forward(self, x):
        range_gates = torch.sigmoid(self.gates)
        for i, layer in enumerate(self.layers):
            x = F.relu(x)
            x = (range_gates[i])*layer(x) + (1-range_gates[i])*x
        return x

    
class Net(torch.nn.Module):
    def __init__(self, init_dim, hidden_dim, hidden_width, layers, p_dropout):
        super().__init__()
        self.dr = nn.Sequential(nn.Linear(init_dim, 512),
                               nn.ReLU(),
                               nn.Dropout(p=p_dropout),
                               nn.Linear(512, hidden_dim))
        # self.ad = nn.Linear(init_dim, hidden_dim)
        self.fc = ResNetGated(hidden_dim, hidden_width, layers, p_dropout)
        self.reg = nn.Linear(hidden_dim, 1)
        self.fc.apply(init_weights)
        self.reg.apply(init_weights)
        self.dr.apply(init_weights)
    def forward(self, x):
        x = self.dr(x)
        x = self.fc(x)
        x = self.reg(x)
        return x

In [17]:
def train(model, device, data ,loss_fn, optimizer):
    model.train()
    train_losses = []
    optimizer.zero_grad()
    for i, batch in enumerate(data):
        data, labels = batch[0].to(device), batch[1][:, 1].to(device)
        logits = model(data)
        loss = loss_fn()
        output=loss(logits.squeeze(), labels.squeeze())
        output.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 3)
        optimizer.step()
        train_loss = output.data.cpu().numpy()
        train_losses.append(train_loss)    
    return np.mean(train_losses)

### Testing function
def test(model, device, data, loss_fn):
    # Set evaluation mode for encoder and decoder
    model.eval()
    test_losses = []
    with torch.no_grad(): # No need to track the gradients
        for i, batch in enumerate(data):
            data, labels = batch[0].to(device), batch[1][:, 1].to(device)
            logits = model(data)
            loss = loss_fn()
            output=loss(logits.squeeze(), labels.squeeze())
            test_loss = output.data.cpu().numpy()
            test_losses.append(test_loss)  
    return np.mean(test_losses)

In [18]:
### Define the loss function
loss_fn = nn.MSELoss

lr= 0.0001
weight_decay = 0.00001
hidden_dim = 256
hidden_width=4096
layers=2
p_dropout = 0.05
### Set the random seed for reproducible results
torch.manual_seed(0)

model = Net(776, hidden_dim, hidden_width, layers, p_dropout=p_dropout)
params_to_optimize = [
    {'params': model.parameters()}
]

optim = torch.optim.Adam(params_to_optimize, lr=lr, weight_decay=weight_decay)
# Check if the GPU is available
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f'Selected device: {device}')

# Move both the encoder and the decoder to the selected device
model.to(device)

Selected device: cuda


Net(
  (dr): Sequential(
    (0): Linear(in_features=776, out_features=512, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.05, inplace=False)
    (3): Linear(in_features=512, out_features=256, bias=True)
  )
  (fc): ResNetGated(
    (layers): ModuleList(
      (0): Sequential(
        (0): Linear(in_features=256, out_features=4096, bias=True)
        (1): ReLU()
        (2): BatchNorm1d(4096, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (3): Linear(in_features=4096, out_features=256, bias=True)
      )
      (1): Sequential(
        (0): Linear(in_features=256, out_features=4096, bias=True)
        (1): ReLU()
        (2): BatchNorm1d(4096, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (3): Linear(in_features=4096, out_features=256, bias=True)
      )
    )
  )
  (reg): Linear(in_features=256, out_features=1, bias=True)
)

In [19]:
import datetime
date = datetime.datetime.today().strftime('%Y-%m-%d-%H:%M:%S')

RUN = date

In [20]:
best_loss = 1000
num_epochs = 100
diz_loss = {'train_loss':[],'val_loss':[]}
decay = 0.98
for epoch in range(num_epochs):
    if epoch == 0:
        for par in optim.param_groups:
            par["lr"] = 0.0001
    elif epoch< 10:
        for par in optim.param_groups:
            par["lr"] = par["lr"] * 1.2
    else:
        for par in optim.param_groups:
            par["lr"] = par["lr"] * decay
    train_loss = train(model, device, train_dataloader, loss_fn, optim)
    test_loss = test(model, device, test_dataloader, loss_fn)
    print('\n EPOCH {}/{} \t train loss {} \t \t val loss {}'.format(epoch + 1, num_epochs, train_loss, test_loss))
    diz_loss['train_loss'].append(train_loss)
    diz_loss['val_loss'].append(test_loss)
    if test_loss < best_loss:
        best_loss = test_loss
        torch.save(model.state_dict(), "./saved_models/type0_{}.pth".format(RUN))


 EPOCH 1/100 	 train loss 2.869351387023926 	 	 val loss 7.418661117553711

 EPOCH 2/100 	 train loss 1.0140750408172607 	 	 val loss 0.692995011806488

 EPOCH 3/100 	 train loss 0.8583161234855652 	 	 val loss 0.7605218291282654

 EPOCH 4/100 	 train loss 0.7969590425491333 	 	 val loss 0.7737590670585632

 EPOCH 5/100 	 train loss 0.7593556642532349 	 	 val loss 0.7756291031837463

 EPOCH 6/100 	 train loss 0.7635230422019958 	 	 val loss 0.7558794617652893

 EPOCH 7/100 	 train loss 0.779250979423523 	 	 val loss 0.753648579120636

 EPOCH 8/100 	 train loss 0.8080943822860718 	 	 val loss 1.0024563074111938

 EPOCH 9/100 	 train loss 0.8185370564460754 	 	 val loss 1.02191960811615

 EPOCH 10/100 	 train loss 0.8330379724502563 	 	 val loss 0.7945025563240051

 EPOCH 11/100 	 train loss 0.7891778945922852 	 	 val loss 0.6570470929145813

 EPOCH 12/100 	 train loss 0.726647138595581 	 	 val loss 0.8419502377510071

 EPOCH 13/100 	 train loss 0.6766954660415649 	 	 val loss 0.8897637

In [22]:
diz_loss = {'train_loss':[],'val_loss':[]}
decay = 0.98
num_epochs = 200
size = 4096
for epoch in range(num_epochs):
    if epoch%50 == 0:
        for par in optim.param_groups:
            par["weight_decay"] = par["weight_decay"]/2
            size = size * 2
            train_dataloader = DataLoader(train_dataset, num_workers=12, batch_size=size, shuffle=True, drop_last=True)
    train_loss = train(model, device, train_dataloader, loss_fn, optim)
    test_loss = test(model, device, test_dataloader, loss_fn)
    print('\n EPOCH {}/{} \t train loss {} \t \t val loss {}'.format(epoch + 1, num_epochs, train_loss, test_loss))
    diz_loss['train_loss'].append(train_loss)
    diz_loss['val_loss'].append(test_loss)
    if test_loss < best_loss:
        best_loss = test_loss
        torch.save(model.state_dict(), "./saved_models/type0_{}.pth".format(RUN))


 EPOCH 1/200 	 train loss 0.03164757788181305 	 	 val loss 0.12122350186109543

 EPOCH 2/200 	 train loss 0.04110248386859894 	 	 val loss 0.10718630999326706

 EPOCH 3/200 	 train loss 0.041913971304893494 	 	 val loss 0.09744340926408768

 EPOCH 4/200 	 train loss 0.05400896072387695 	 	 val loss 0.1546802520751953

 EPOCH 5/200 	 train loss 0.0681627094745636 	 	 val loss 0.1799907237291336

 EPOCH 6/200 	 train loss 0.05198962241411209 	 	 val loss 0.10263674706220627

 EPOCH 7/200 	 train loss 0.03865962475538254 	 	 val loss 0.08727234601974487

 EPOCH 8/200 	 train loss 0.03873463720083237 	 	 val loss 0.14643220603466034

 EPOCH 9/200 	 train loss 0.04965275153517723 	 	 val loss 0.11107394844293594

 EPOCH 10/200 	 train loss 0.04433818534016609 	 	 val loss 0.12910117208957672

 EPOCH 11/200 	 train loss 0.0633210688829422 	 	 val loss 0.10332249850034714

 EPOCH 12/200 	 train loss 0.055740103125572205 	 	 val loss 0.11907247453927994

 EPOCH 13/200 	 train loss 0.056131847

In [23]:
decay = 0.98
num_epochs = 500
size = 4096
for epoch in range(num_epochs):
    # if epoch%50 == 0:
    #     for par in optim.param_groups:
    #         par["weight_decay"] = par["weight_decay"]/2
    #         size = size * 2
    #         train_dataloader = DataLoader(train_dataset, num_workers=12, batch_size=size, shuffle=True, drop_last=True)
    train_loss = train(model, device, train_dataloader, loss_fn, optim)
    test_loss = test(model, device, test_dataloader, loss_fn)
    print('\n EPOCH {}/{} \t train loss {} \t \t val loss {}'.format(epoch + 1, num_epochs, train_loss, test_loss))
    diz_loss['train_loss'].append(train_loss)
    diz_loss['val_loss'].append(test_loss)
    if test_loss < best_loss:
        best_loss = test_loss
        torch.save(model.state_dict(), "./saved_models/type0_{}.pth".format(RUN))


 EPOCH 1/500 	 train loss 0.010452802293002605 	 	 val loss 0.07303854078054428

 EPOCH 2/500 	 train loss 0.010427760891616344 	 	 val loss 0.07228532433509827

 EPOCH 3/500 	 train loss 0.010319340042769909 	 	 val loss 0.07227735221385956

 EPOCH 4/500 	 train loss 0.010482829064130783 	 	 val loss 0.07252863794565201

 EPOCH 5/500 	 train loss 0.01042421068996191 	 	 val loss 0.07238736748695374

 EPOCH 6/500 	 train loss 0.010324394330382347 	 	 val loss 0.07259107381105423

 EPOCH 7/500 	 train loss 0.010329483076930046 	 	 val loss 0.07316642254590988

 EPOCH 8/500 	 train loss 0.010620188899338245 	 	 val loss 0.07190406322479248

 EPOCH 9/500 	 train loss 0.010318691842257977 	 	 val loss 0.07240932434797287

 EPOCH 10/500 	 train loss 0.010461919009685516 	 	 val loss 0.07274122536182404

 EPOCH 11/500 	 train loss 0.01036594994366169 	 	 val loss 0.07242170721292496

 EPOCH 12/500 	 train loss 0.01044019777327776 	 	 val loss 0.07219108194112778

 EPOCH 13/500 	 train loss 

In [24]:
print(RUN)

2022-02-05-21:29:34
